In [1]:
!pip install datasets transformers evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.5 MB/s eta 0:00:00


In [1]:
from datasets import load_dataset

langs = [ "de", "en", "es", "hi"]
translate_langs = [ "de",  "es"]
langs_test = []
langs_translate_test = []
langs_translate_train = [ "de.de", "en.en", "es.es", "hi.hi"]
langs_translate_train_all = []

mlqa = {}

for lang1 in langs:
    for lang2 in langs:
        mlqa[f"{lang1}.{lang2}"] = load_dataset("mlqa", f"mlqa.{lang1}.{lang2}")
        langs_test.append(f"{lang1}.{lang2}")

for lang in translate_langs:
    mlqa[f"translate-train.{lang}"] = load_dataset("mlqa", f"mlqa-translate-train.{lang}")
    mlqa[f"translate-test.{lang}"] = load_dataset("mlqa", f"mlqa-translate-test.{lang}")
    langs_translate_test.append(f"translate-test.{lang}")
    langs_translate_train_all.append(f"translate-train.{lang}")

In [16]:
mlqa['hi.en']['validation'][0]

{'context': 'पैरेनकाइमा कोशिकाएं वे जीवित कोशिकाएं हैं जो संग्रह और सहारे से लेकर प्रकाश संश्लेषण और फ्लोएम लोडिंग तक के विभिन्न कार्य करती हैं। जाइलेम और फ्लोएम के अलावा पत्तियां मुख्यतः पैरेन्काइमा कोशिकाओं से बनी होती हैं। बाह्यत्वचा की तरह कुछ पैरेन्काइमा कोशिकाएं प्रकाश भेद्यता और गैस विनिमय के नियंत्रण की निशिष्यता रखती हैं, पर अन्य पौधे के ऊतक की सबसे कम विशेषज्ञ कोशिकाएं होती हैं और टोटीपोटेंट बनी रहती हैं जो जीवनपर्यंत विभाजित होकर नई अविभेदित कोशिकाओं का उत्पादन करती हैं। पैरेन्काइमा कोशिकाओं की प्राथमिक भित्तियां पतली और पारगम्य होती हैं जिनमें से छोटे अणुओं का परिवहन हो सकता है और उनका साइटोप्लाज्म विभिन्न जैवरसायनिक कार्यकलापों जैसे नेक्टर के स्राव या पत्ते खाने की आदत को अनुत्साहित करने वाले द्वितीयक पदार्थों का उत्पादन करने के लिये जिम्मेदार होता है। वे पैरेन्काइमा कोशिकाएं जिनमें कई क्लोरोप्लास्ट होते हैं और प्रकाश संश्लेषण का कार्य करते हैं उन्हें क्लोरेन्काइमा कोशिकाएं कहते हैं। आलू जैसे ट्यूबरों और फलियों के बीज कॉटीलीडॉनों का काम संग्रह करना होता है।कोलेनकाइमा कोशिक

In [4]:
max_length = 384 
doc_stride = 128 

In [4]:
def prepare_train_features(examples):
    examples["question"] = [q.lstrip() for q in examples["question"]]
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    
    
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    
    
    offset_mapping = tokenized_examples.pop("offset_mapping")

    
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        
        sequence_ids = tokenized_examples.sequence_ids(i)

        
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                
                
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [6]:
def prepare_validation_features(examples):
    
    examples["question"] = [q.lstrip() for q in examples["question"]]


    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    
    
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1
        
        
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        
        
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [7]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [8]:
from tqdm.auto import tqdm
import collections
import numpy as np

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    
    predictions = collections.OrderedDict()

    
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    
    for example_index, example in enumerate(tqdm(examples)):
        
        feature_indices = features_per_example[example_index]

        valid_answers = []
        
        context = example["context"]
        
        for feature_index in feature_indices:
            
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            
            
            offset_mapping = features[feature_index]["offset_mapping"]

            
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    
                    
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            
            
            best_answer = {"text": "", "score": 0.0}
        
        
        predictions[example["id"]] = best_answer["text"]

    return predictions

In [9]:
from collections import defaultdict

mlqa_prep = defaultdict(dict)

def map_datasets(langs, split, prepare_features):
    for lang in langs:
        mlqa_prep[lang][split] = mlqa[lang][split].map(prepare_features, batched=True, 
                                    remove_columns=mlqa[lang][split].column_names)

In [11]:
from evaluate import load

squad_metric = load("squad")

def compute_results(langs, split):
    results = {}
    for lang in langs:
        
        raw_predictions = trainer.predict(mlqa_prep[lang][split])

        
        mlqa_prep[lang][split].set_format(type=mlqa_prep[lang][split].format["type"], 
                        columns=list(mlqa_prep[lang][split].features.keys()))
        
        
        final_predictions = postprocess_qa_predictions(mlqa[lang][split], mlqa_prep[lang][split], raw_predictions.predictions)

        
        formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
        references = [{"id": ex["id"], "answers": ex["answers"]} for ex in mlqa[lang][split]]
        results[lang] = squad_metric.compute(predictions=formatted_predictions, references=references)
    return results

In [12]:
import pandas as pd

def results_df(results_dict, model):
    F1colname = "F1_" + model
    EMcolname = "EM_" + model
    dict_results = defaultdict(list)
    for lang, scores in results_dict.items():
        dict_results["lang"].append(lang)
        dict_results[F1colname].append(scores['f1'])
        dict_results[EMcolname].append(scores['exact_match'])

    avg_f1 = np.average(dict_results[F1colname])
    avg_em = np.average(dict_results[EMcolname])
    dict_results["lang"].append('avg')
    dict_results[F1colname].append(avg_f1)
    dict_results[EMcolname].append(avg_em)
    df_results = pd.DataFrame(dict_results).round(2)
    return df_results

In [15]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments, Trainer, DefaultDataCollator

model_name = "alon-albalak/bert-base-multilingual-xquad"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
data_collator = DefaultDataCollator()

batch_size = 16
training_args = TrainingArguments(
    output_dir="bert-base-multilingual-xquad",
    evaluation_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.03,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=None,
    eval_dataset=None,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

config.json:   0%|          | 0.00/819 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/667M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
langs = ["de.de", "en.en", "es.es", "hi.hi"]
split = "test"

map_datasets(langs, split, prepare_validation_features)

Map:   0%|          | 0/4517 [00:00<?, ? examples/s]

Map:   0%|          | 0/11590 [00:00<?, ? examples/s]

Map:   0%|          | 0/5253 [00:00<?, ? examples/s]

Map:   0%|          | 0/4918 [00:00<?, ? examples/s]

In [37]:
results_fine_tuning_xquad_mbert = compute_results(langs, split)
print(results_fine_tuning_xquad_mbert)

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):


Post-processing 4517 example predictions split into 5191 features.


  0%|          | 0/4517 [00:00<?, ?it/s]

Post-processing 11590 example predictions split into 14522 features.


  0%|          | 0/11590 [00:00<?, ?it/s]

Post-processing 5253 example predictions split into 5432 features.


  0%|          | 0/5253 [00:00<?, ?it/s]

Post-processing 4918 example predictions split into 6982 features.


  0%|          | 0/4918 [00:00<?, ?it/s]

{'de.de': {'exact_match': 36.90502545937569, 'f1': 53.26384255841377}, 'en.en': {'exact_match': 51.83779119930975, 'f1': 66.42193987369646}, 'es.es': {'exact_match': 36.03655054254712, 'f1': 56.93500029971474}, 'hi.hi': {'exact_match': 31.496543310288736, 'f1': 47.61393239713055}}


In [43]:
df_results_fine_tuning_xquad_mbert = results_df(results_fine_tuning_xquad_mbert, "FT_xquad_mbert")
df_results_fine_tuning_xquad_mbert.to_csv("results_fine_tuning_xquad_mbert.csv")
df_results_fine_tuning_xquad_mbert

lang  F1_FT_xquad_mbert  EM_FT_xquad_mbert
0  de.de              53.26              36.91
1  en.en              66.42              51.84
2  es.es              56.94              36.04
3  hi.hi              47.61              31.50
4    avg              56.06              39.07

In [45]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments, Trainer, DefaultDataCollator

model_name = "alon-albalak/xlm-roberta-base-xquad"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
data_collator = DefaultDataCollator()

batch_size = 16
training_args = TrainingArguments(
    output_dir="xlm-roberta-base-xquad",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.02,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=None,
    eval_dataset=None,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

pytorch_model.bin:  26%|##5       | 283M/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/356 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [46]:
langs = ["de.de", "en.en", "es.es", "hi.hi"]
split = "test"

map_datasets(langs, split, prepare_validation_features)

Map:   0%|          | 0/4517 [00:00<?, ? examples/s]

Map:   0%|          | 0/11590 [00:00<?, ? examples/s]

Map:   0%|          | 0/5253 [00:00<?, ? examples/s]

Map:   0%|          | 0/4918 [00:00<?, ? examples/s]

In [47]:
results_fine_tuning_xquad_xlm_r = compute_results(langs, split)
print(results_fine_tuning_xquad_xlm_r)

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):


Post-processing 4517 example predictions split into 5278 features.


  0%|          | 0/4517 [00:00<?, ?it/s]

Post-processing 11590 example predictions split into 15269 features.


  0%|          | 0/11590 [00:00<?, ?it/s]

Post-processing 5253 example predictions split into 5457 features.


  0%|          | 0/5253 [00:00<?, ?it/s]

Post-processing 4918 example predictions split into 6351 features.


  0%|          | 0/4918 [00:00<?, ?it/s]

{'de.de': {'exact_match': 36.04162054460925, 'f1': 51.73250853768612}, 'en.en': {'exact_match': 53.011216566005174, 'f1': 66.80334369561412}, 'es.es': {'exact_match': 37.02646106986484, 'f1': 57.774367073867175}, 'hi.hi': {'exact_match': 36.11224074827165, 'f1': 52.85778364516253}}


In [49]:
df_results_fine_tuning_xquad_xlm_r = results_df(results_fine_tuning_xquad_xlm_r, "FT_xquad_xlm_r")
df_results_fine_tuning_xquad_xlm_r.to_csv("results_fine_tuning_xquad_xlm_r.csv")
df_results_fine_tuning_xquad_xlm_r

lang  F1_FT_xquad_xlm_r  EM_FT_xquad_xlm_r
0  de.de              51.73              36.04
1  en.en              66.80              53.01
2  es.es              57.77              37.03
3  hi.hi              52.86              36.11
4    avg              57.29              40.55

In [50]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments, Trainer, DefaultDataCollator

model_name = "alon-albalak/xlm-roberta-large-xquad"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
data_collator = DefaultDataCollator()

batch_size = 16
training_args = TrainingArguments(
    output_dir="xlm-roberta-large-xquad",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.02,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=None,
    eval_dataset=None,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [51]:
langs = ["de.de", "en.en", "es.es", "hi.hi"]
split = "test"

map_datasets(langs, split, prepare_validation_features)

Map:   0%|          | 0/4517 [00:00<?, ? examples/s]

Map:   0%|          | 0/11590 [00:00<?, ? examples/s]

Map:   0%|          | 0/5253 [00:00<?, ? examples/s]

Map:   0%|          | 0/4918 [00:00<?, ? examples/s]

In [52]:
results_fine_tuning_xquad_xlm_r_large = compute_results(langs, split)
print(results_fine_tuning_xquad_xlm_r_large)

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):


Post-processing 4517 example predictions split into 5278 features.


  0%|          | 0/4517 [00:00<?, ?it/s]

Post-processing 11590 example predictions split into 15269 features.


  0%|          | 0/11590 [00:00<?, ?it/s]

Post-processing 5253 example predictions split into 5457 features.


  0%|          | 0/5253 [00:00<?, ?it/s]

Post-processing 4918 example predictions split into 6351 features.


  0%|          | 0/4918 [00:00<?, ?it/s]

{'de.de': {'exact_match': 44.71994686738986, 'f1': 61.406792323161}, 'en.en': {'exact_match': 61.44089732528042, 'f1': 75.56991243830646}, 'es.es': {'exact_match': 43.7845040928993, 'f1': 65.12533739178873}, 'hi.hi': {'exact_match': 45.24196827978853, 'f1': 63.09591529038426}}


In [53]:
df_results_fine_tuning_xquad_xlm_r_large = results_df(results_fine_tuning_xquad_xlm_r_large, "FT_xquad_xml_r_large")
df_results_fine_tuning_xquad_xlm_r_large.to_csv("results_fine_tuning_xquad_xlm_r_large.csv")
df_results_fine_tuning_xquad_xlm_r_large

lang  F1_FT_xquad_xml_r_large  EM_FT_xquad_xml_r_large
0  de.de                    61.41                    44.72
1  en.en                    75.57                    61.44
2  es.es                    65.13                    43.78
3  hi.hi                    63.10                    45.24
4    avg                    66.30                    48.80

In [54]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments, Trainer, DefaultDataCollator

model_name = "mrm8488/bert-multi-cased-finetuned-xquadv1"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
data_collator = DefaultDataCollator()

batch_size = 16
training_args = TrainingArguments(
    output_dir="bert-multi-cased-finetuned-xquadv1",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.02,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=None,
    eval_dataset=None,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

config.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

Some weights of the model checkpoint at mrm8488/bert-multi-cased-finetuned-xquadv1 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [55]:
langs = ["de.de", "en.en", "es.es", "hi.hi"]
split = "test"

map_datasets(langs, split, prepare_validation_features)

Map:   0%|          | 0/4517 [00:00<?, ? examples/s]

Map:   0%|          | 0/11590 [00:00<?, ? examples/s]

Map:   0%|          | 0/5253 [00:00<?, ? examples/s]

Map:   0%|          | 0/4918 [00:00<?, ? examples/s]

In [56]:
results_data_augmentation_mbert = compute_results(langs, split)
print(results_data_augmentation_mbert)

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):


Post-processing 4517 example predictions split into 5242 features.


  0%|          | 0/4517 [00:00<?, ?it/s]

Post-processing 11590 example predictions split into 14706 features.


  0%|          | 0/11590 [00:00<?, ?it/s]

Post-processing 5253 example predictions split into 5456 features.


  0%|          | 0/5253 [00:00<?, ?it/s]

Post-processing 4918 example predictions split into 7500 features.


  0%|          | 0/4918 [00:00<?, ?it/s]

{'de.de': {'exact_match': 38.166924950188175, 'f1': 54.30902615680056}, 'en.en': {'exact_match': 53.3304572907679, 'f1': 67.59029272229766}, 'es.es': {'exact_match': 38.37806967447173, 'f1': 58.99040442086852}, 'hi.hi': {'exact_match': 34.8922326148841, 'f1': 50.96694383958836}}


In [57]:
df_results_data_augmentation_mbert = results_df(results_data_augmentation_mbert, "data_augm_mbert")
df_results_data_augmentation_mbert.to_csv("results_data_augmentation_mbert.csv")
df_results_data_augmentation_mbert

lang  F1_data_augm_mbert  EM_data_augm_mbert
0  de.de               54.31               38.17
1  en.en               67.59               53.33
2  es.es               58.99               38.38
3  hi.hi               50.97               34.89
4    avg               57.96               41.19